##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/get_started"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 ai.google.dev 上查看</a>
  </td>
    <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/gemma/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/google/generative-ai-docs/main/site/en/gemma/docs/get_started.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />在 Vertex AI 中打開</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/gemma/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視來源</a>
  </td>
</table>


# 使用 KerasNLP 開始使用 Gemma

本教學說明如何使用 [KerasNLP](https://keras.io/keras_nlp/) 開始使用 Gemma。Gemma 是一系列輕量級、最先進的開放模型，由建置雙子座模型時所使用的相同研究和技術所建置而成。KerasNLP 是一系列自然語言處理 (NLP) 模型，在 [Keras](https://keras.io/) 中實作，可在 JAX、PyTorch 和 TensorFlow 上執行。

在本教學中，你將使用 Gemma 來產生對數個提示的文字回應。如果你是 Keras 新手，你可能希望在開始之前閱讀 [開始使用 Keras](https://keras.io/getting_started/)，但這不是必要的。你會在完成本教學時進一步了解 Keras。


## 設定


### Gemma 設定

若要完成本教學課程，你首先需要完成 [Gemma 設定](https://ai.google.dev/gemma/docs/setup) 的設定說明。Gemma 設定說明會說明如何進行下列作業：

* 在 kaggle.com 上取得 Gemma 的存取權限。
* 選取資源足夠、可以執行 Gemma 2B 模型的 Colab 執行環境。
* 產生並設定 Kaggle 使用者名稱和 API 金鑰。

完成 Gemma 設定後，移至下一部分，你將設定 Colab 環境的環境變數。


### 設定環境變數

為 `KAGGLE_USERNAME` 和 `KAGGLE_KEY` 設定環境變數。


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### 安裝依賴項

安裝 Keras 和 KerasNLP。


In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

### 選擇後端

Keras 是高階、多框架深度學習 API，設計用於簡化和易於使用。[Keras 3](https://keras.io/keras_3) 讓你選擇後端：TensorFlow、JAX 或 PyTorch。這三個都適用於本課程。


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

### 匯入套件

匯入 Keras 和 KerasNLP。


In [ ]:
import keras
import keras_nlp

## 建立模型

KerasNLP 提供許多熱門 [模型結構](https://keras.io/api/keras_nlp/models/) 實作。在本教學中，你將使用 `GemmaCausalLM` 建立一個模型，這是一個端對端的 Gemma 模型，可用於因果語言建模。因果語言模型可根據之前的 Token 預測下一個 Token。

使用 `from_preset` 方法建立模型：


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")


Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...


`from_preset` 透過預設架構和權重實例化模型。在以上的程式碼中，字串 `"gemma_2b_en"` 指出預設架構：一個有 20 億個參數的 Gemma 模型。


註：我們也提供一個參數為 70 億的 Gemma 模型。若要在 Colab 中執行較大的模型，你需要使用付費方案中提供的焦點 GPU。你也可以在 Kaggle 或 Google Cloud 上對 Gemma 7B 模型執行 [分散式調整](https://ai.google.dev/gemma/docs/distributed_tuning)。


使用 `summary` 取得更多模型資訊：


In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

正如你從摘要中看到的那樣，該模型具有 25 億個可訓練參數。

註：為命名模型 (「2B」) 之目的，嵌入層未計入參數數量。


## 產生文字

現在是產生一些文字的時候了！模型具有可以根據提示產生文字的 `generate` 方法。可選的 `max_length` 引數指定產生的序列之最大長度。

使用提示 `"生命的意義是什麼？"` 試試看。


In [ ]:
gemma_lm.generate("What is the meaning of life?", max_length=64)

'What is the meaning of life?\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been asked by people who are looking for answers to their own lives'

再試著用不同的提示詞呼叫「產生」。


In [ ]:
gemma_lm.generate("How does the brain work?", max_length=64)

'How does the brain work?\n\nThe brain is the most complex organ in the human body. It is responsible for controlling all of the body’s functions, including breathing, heart rate, digestion, and more. The brain is also responsible for thinking, feeling, and making decisions.\n\nThe brain is made up'

如果你在 JAX 或 TensorFlow 後端執行，你將會注意到第二次 `generate` 呼叫幾乎立刻傳回。這是因為針對給定批次大小和 `max_length` 的每次 `generate` 呼叫都是使用 XLA 編譯的。第一次執行成本高，但後續執行快很多。


你也可以提供分批提示，使用一個作為輸入的列表：


In [ ]:
gemma_lm.generate(
    ["What is the meaning of life?",
     "How does the brain work?"],
    max_length=64)

['What is the meaning of life?\n\nThe question is one of the most important questions in the world.\n\nIt’s the question that has been asked by philosophers, theologians, and scientists for centuries.\n\nAnd it’s the question that has been asked by people who are looking for answers to their own lives',
 'How does the brain work?\n\nThe brain is the most complex organ in the human body. It is responsible for controlling all of the body’s functions, including breathing, heart rate, digestion, and more. The brain is also responsible for thinking, feeling, and making decisions.\n\nThe brain is made up']

### 可選：嘗試不同的取樣器

你可以透過在「compile()」中設定「sampler」引數來控制「GemmaCausalLM」的生成策略。預設情況下，將使用「greedy」取樣。

作為一種實驗，嘗試設定「top_k」策略：


In [ ]:
gemma_lm.compile(sampler="top_k")
gemma_lm.generate("What is the meaning of life?", max_length=64)

'What is the meaning of life? That was a question I asked myself as I was driving home from work one night in 2012. I was driving through the city of San Bernardino, and all I could think was, “What the heck am I doing?”\n\nMy life was completely different. I'

雖然預設的貪婪演算法總是選擇機率最大的 Token，但 Top K 演算法會從前 K 大機率的 Token 中隨機選擇下一個 Token。

你不必指定一個取樣器，並且如果你用不到最後一個程式碼片段，你可以忽略它。如果你想要進一步了解可用的取樣器，請參閱 [Samplers](https://keras.io/api/keras_nlp/samplers/)。


## 接下來是什麼

在本教學課程中，瞭解如何使用 KerasNLP 和 Gemma 產生文字。以下是學習下一階段內容的一些建議：

* 瞭解如何 [微調 Gemma 模型](https://ai.google.dev/gemma/docs/lora_tuning)。
* 瞭解如何在 Gemma 模型上進行 [分散式微調和推論](https://ai.google.dev/gemma/docs/distributed_tuning)。
* 瞭解 [Gemma 與 Vertex AI 的整合](https://ai.google.dev/gemma/docs/integrations/vertex)
* 瞭解如何 [使用 Gemma 模型與 Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma)。
